# Import

In [55]:
import pandas as pd

In [56]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [57]:
train_df = pd.read_csv("../../data/train.csv")
test_df = pd.read_csv("../../data/test.csv")  
submission = pd.read_csv("../../data/sample_submission.csv")  

In [58]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54609 non-null  object
 1   분류      54609 non-null  object
 2   제목      54609 non-null  object
 3   키워드     54609 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [59]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
dtypes: object(3)
memory usage: 548.7+ KB


In [60]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   분류      23405 non-null  object
dtypes: object(2)
memory usage: 365.8+ KB


In [61]:
len(train_df['분류'].value_counts())

56

타겟변수 -> '분류'이고 종류 56가지

In [62]:
# %pip install konlpy

In [63]:
from konlpy.tag import Okt

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 제외할 품사 목록 정의
'''
Suffix: 접미사
Determiner: 관형사
Adverb: 부사
Conjunction: 접속사
Josa: 조사
PreEomi: 선어말 어미
Eomi: 어미
Punctuation: 구두점
Foreign: 외국어
Alpha: 알파벳
Number: 숫자
Unknown: 알 수 없는 품사
'''

exclude_pos = ['Suffix', 'Determiner', 'Adverb', 'Conjunction', 'Josa', 'PreEomi', 'Eomi', 'Punctuation', 'Foreign', 'Alpha', 'Number', 'Unknown']

# '제목' 열에서 일반명사와 고유명사만 추출하여 '키워드_추출' 열에 저장하는 함수
def extract_nouns_and_proper_nouns_from_title(title):
    words = okt.pos(title)
    nouns = [word for word, pos in words if pos in ['Noun', 'ProperNoun'] and word not in exclude_pos]
    return ', '.join(nouns)

# '제목' 열에 함수 적용하여 '키워드_추출' 열 생성
train_df['키워드_추출'] = train_df['제목'].apply(extract_nouns_and_proper_nouns_from_title)
test_df['키워드_추출'] = test_df['제목'].apply(extract_nouns_and_proper_nouns_from_title)

# 결과 출력
print(train_df[['제목', '키워드_추출']].head(10))

                                               제목  \
0           용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최   
1                    용인 농촌테마파크, 7~8월 단체체험객 체험료 지원   
2                 용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장   
3                 수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대   
4               용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할   
5                      소강석 목사 "한미 참전용사 끝까지 찾아뵐 것"   
6  [지방선거 D-360] 전 현직 공직출신 후보군, 행정경험 무기로 '단체장' 노린다   
7                    용인시, '자동차세 연납제도'로 9.15% 세액공제   
8             울진 '원자력수소 국가산단' 행정절차 '속속'...기본협약 체결   
9                 용인시, ‘용인-화성 광역 버스정보시스템 구축사업’ 착수   

                                         키워드_추출  
0          용인, 문화재단, 인문학, 콘서트, 당신, 뮤지컬, 이야기, 개최  
1          용인, 농촌, 테마, 파크, 단체, 체험, 객, 체험, 료, 지원  
2              용인시, 노후, 주택, 에너지, 성능, 개선, 신청, 연장  
3      수원, 용인, 고양시, 특례시, 로, 지정, 도시, 경쟁력, 증가, 기대  
4  용인시, 스페인, 미국, 국제, 명예, 문관, 위촉, 대외, 홍보, 지원, 역할  
5                       소강석, 목사, 한미, 참전용사, 끝, 것  
6       지방선거, 전, 현직, 공직, 출신, 후보, 행정, 경험, 무기, 단체  
7               

In [37]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54609 non-null  object
 1   분류      54609 non-null  object
 2   제목      54609 non-null  object
 3   키워드     54609 non-null  object
 4   키워드_추출  54609 non-null  object
dtypes: object(5)
memory usage: 2.1+ MB


In [38]:
# 키워드와 키워드_추출 값을 합쳐서 새로운 변수 생성
train_df['키워드_통합'] = train_df['키워드'] + ' ' + train_df['키워드_추출']
test_df['키워드_통합'] = test_df['키워드'] + ' ' + test_df['키워드_추출']

# 변수 드랍
train_df = train_df.drop(columns=['제목', '키워드', '키워드_추출'])
test_df = test_df.drop(columns=['제목', '키워드', '키워드_추출'])

# 결과 확인
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54609 non-null  object
 1   분류      54609 non-null  object
 2   키워드_통합  54609 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [39]:
train_df.head()

,ID,분류,키워드_통합
0,TRAIN_00000,문화:전시_공연,"용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코..."


In [40]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   키워드_통합  23405 non-null  object
dtypes: object(2)
memory usage: 365.8+ KB


In [41]:
test_df.head()

,ID,키워드_통합
0,TEST_00000,"김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,이노코리아,대표,희정,한성대..."
1,TEST_00001,"신규,확진,나흘,세자릿수,방역당국,핼러윈,풍선,효과,차단,총력,감염증,신종,코로나바..."
2,TEST_00002,"전해철,장관,재정,분권,강화,지방자치,2.0,시대,마중물,마련,장관,전해철,행정안전..."
3,TEST_00003,"용인시,구인,장애인,구직,만남,채용,행사,노호근,용인특례시,장애인,취업,지원,대회의..."
4,TEST_00004,"지자체,경기,북동부,지역,산업단지,혁신단위,설정,전략,지역,연계,특성,제시,경기도경..."


## 전처리

In [42]:
import re
from collections import Counter

# 불용어 제거
removed_keywords = []

def remove_invalid_keywords(keywords):
    pattern = re.compile(
        r'[A-Za-z]+[가-힣\u4E00-\u9FFF]+|'  # 영어+한글
        r'[0-9]+[가-힣\u4E00-\u9FFF]+|'     # 숫자+한글
        r'[0-9]+[A-Za-z]+|'                 # 숫자+영어
        r'[가-힣]+[A-Za-z\u4E00-\u9FFF]+|'  # 한글+영어
        r'[가-힣]+[0-9]+|'                  # 한글+숫자
        r'[A-Za-z]+[0-9]+|'                 # 영어+숫자
        r'[\u4E00-\u9FFF]+|'                # 한자
        r'[0-9]+(\.[0-9]+)?%|'              # 숫자+퍼센트
        r'[0-9]+|'                          # 숫자
        r'[A-Za-z]+'                        # 영어
    )
    valid_keywords = []
    for word in keywords.split(','):
        word = word.strip()
        if not word or pattern.match(word):  # 공백이거나 패턴에 맞는 단어 제거
            removed_keywords.append(word)
        else:
            valid_keywords.append(word)
    return ', '.join(valid_keywords)

# 원본 데이터에서 해당 키워드들을 제거
train_df['키워드_통합'] = train_df['키워드_통합'].apply(remove_invalid_keywords)
test_df['키워드_통합'] = test_df['키워드_통합'].apply(remove_invalid_keywords)

# 결과 출력
print("\n수정된 데이터프레임:")
print(train_df)

# 제거된 단어들 출력
print("\n제거된 단어들:")
print(removed_keywords[:10])


수정된 데이터프레임:
                ID        분류  \
0      TRAIN_00000  문화:전시_공연   
1      TRAIN_00001        지역   
2      TRAIN_00002        지역   
3      TRAIN_00003        지역   
4      TRAIN_00004        국제   
...            ...       ...   
54604  TRAIN_54604        국제   
54605  TRAIN_54605  사회:교육_시험   
54606  TRAIN_54606        지역   
54607  TRAIN_54607        지역   
54608  TRAIN_54608        지역   

                                                  키워드_통합  
0      용인문화재단, 인문학, 콘서트, 뮤지컬, 이야기, 개최, 인문학, 콘서트, 뮤지컬,...  
1      용인, 농촌, 테마파크, 단체, 체험객, 체험료, 지원, 체험일, 기준, 용인시통합...  
2      용인시, 노후, 주택, 에너지, 성능, 개선, 신청, 연장, 용인시청, 용인시, 노...  
3      수원, 용인, 고양시, 특례시, 지정, 도시경쟁력, 증가, 경기, 도내, 인구, 수...  
4      용인시, 스페인, 미국, 국제, 명예, 자문관, 위촉, 역할, 대외, 홍보, 지원,...  
...                                                  ...  
54604  용인, 아파트, 여성, 아들, 추락, 극단, 선택, 추정, 경찰, 현장, 유서, 경...  
54605  용인시, 위탁, 부모, 보수, 교육, 용인시, 경기, 남부, 가정, 위탁, 지원, ...  
54606  용인시, 플랫폼, 시티, 국토부, 신청, 사업, 인정, 협의, 경기, 용인시, 중앙...  
54607  주민자치위

In [43]:
# 제거된 단어들의 빈도 계산 및 상위 20개 출력
keyword_counter = Counter(removed_keywords)
top_10_keywords = keyword_counter.most_common(20)

print("\n제거된 단어들 상위 20개:")
for keyword, count in top_10_keywords:
    print(f"{keyword}: {count}")


제거된 단어들 상위 20개:
코로나19: 20593
A씨: 13183
Yongin: 11405
City: 7544
1만: 6429
city: 5846
: 5726
SK하이닉스: 5603
AI: 5271
Google: 5108
SK: 4812
1년: 4811
GTX: 4286
LH: 4246
Translate: 4229
Lee: 4051
3년: 4024
2년: 3938
Gyeonggi: 3858
B씨: 3769


## 대분류 기준으로 공통단어 제거

In [44]:
# '분류' 열에서 앞부분만 추출하여 '분류_대분류'라는 새로운 열에 저장
train_df['분류_대분류'] = train_df['분류'].apply(lambda x: x.split(':')[0])

# 결과 확인
train_df['분류_대분류'].value_counts()

지역       26950
경제       10534
사회        8245
정치        2521
문화        2500
스포츠       2035
IT_과학     1487
국제         337
Name: 분류_대분류, dtype: int64

In [45]:
len(train_df)

54609

In [47]:
from collections import Counter

def find_common_words_and_remove(num_categories=8, top_n=100, com_counts=10, train_df=train_df, test_df=test_df):
    # '분류_대분류'의 각 범주에 속하는 단어들을 추출
    category_words = {category: [] for category in train_df["분류_대분류"].unique()}

    for category in category_words.keys():
        words = train_df.loc[train_df["분류_대분류"] == category, "키워드_통합"].apply(lambda x: x.split(',')).tolist()
        category_words[category] = [word for sublist in words for word in sublist]

    # 각 범주에 속하는 단어들을 카운팅
    word_counts = {category: Counter(words) for category, words in category_words.items()}

    # 지정된 범주 갯수에 속하는 단어들을 찾기
    common_words = set()
    for word in word_counts[list(word_counts.keys())[0]].keys():
        count = sum(1 for category in word_counts.keys() if word in word_counts[category] and word_counts[category][word] >= com_counts)
        if count == num_categories:
            common_words.add(word)

    # 지정된 범주 갯수에 속하는 단어와 그 갯수를 계산
    common_word_counts = {word: sum(word_counts[category][word] for category in word_counts.keys()) for word in common_words}

    # 단어들을 갯수 기준으로 정렬하고 상위 N개를 선택
    top_common_words = sorted(common_word_counts.items(), key=lambda x: x[1], reverse=True)[:top_n]
    print(f"상위 {top_n}개의 단어 확인")

    # 상위 N개의 단어와 그 갯수, 그리고 각 범주에서의 갯수를 출력
    for word, total_count in top_common_words:
        category_counts = {category: word_counts[category][word] for category in word_counts.keys()}
        print(f"단어: {word}, 총 갯수: {total_count}, 각 범주에서의 갯수: {category_counts}")
        
    # 총 몇 개의 단어가 겹치는지 출력
    print(f"총 {len(common_words)}개의 단어가 겹칩니다.")

    # top_n 기준으로 겹치는 단어들을 '키워드' 열에서 제거하고 새로운 열에 저장
    top_common_words_set = set(word for word, _ in top_common_words)
    
    def remove_top_common_words(keywords):
        return ', '.join([word for word in keywords.split(',') if word not in top_common_words_set])

    train_df['키워드_통합'] = train_df['키워드_통합'].apply(remove_top_common_words)
    test_df['키워드_통합'] = test_df['키워드_통합'].apply(remove_top_common_words)

num_categories : 공통단어로 간주할 범주의 수  
top_n : 상위 N개의 공통 단어를 선택  
com_counts : 범주에서 단어가 공통 단어로 간주되기 위해 나타나야 하는 최소 횟수를 지정  

In [48]:
find_common_words_and_remove(num_categories=8, com_counts=100) # top_n은 default인 100 그대로 사용

상위 100개의 단어 확인
단어:  용인시, 총 갯수: 94682, 각 범주에서의 갯수: {'문화': 3729, '지역': 51521, '국제': 471, '정치': 4083, '경제': 17801, '사회': 11619, '스포츠': 2661, 'IT_과학': 2797}
단어:  용인, 총 갯수: 70878, 각 범주에서의 갯수: {'문화': 3573, '지역': 36190, '국제': 266, '정치': 3974, '경제': 20278, '사회': 4149, '스포츠': 1250, 'IT_과학': 1198}
단어:  지역, 총 갯수: 53726, 각 범주에서의 갯수: {'문화': 1400, '지역': 29437, '국제': 131, '정치': 3238, '경제': 13148, '사회': 5264, '스포츠': 183, 'IT_과학': 925}
단어:  지원, 총 갯수: 53337, 각 범주에서의 갯수: {'문화': 533, '지역': 32280, '국제': 105, '정치': 1795, '경제': 13172, '사회': 4118, '스포츠': 252, 'IT_과학': 1082}
단어:  시장, 총 갯수: 49593, 각 범주에서의 갯수: {'문화': 1643, '지역': 25853, '국제': 327, '정치': 4068, '경제': 13638, '사회': 2286, '스포츠': 811, 'IT_과학': 967}
단어:  경기도, 총 갯수: 45053, 각 범주에서의 갯수: {'문화': 1278, '지역': 25564, '국제': 126, '정치': 2357, '경제': 9318, '사회': 3970, '스포츠': 1867, 'IT_과학': 573}
단어:  경기, 총 갯수: 39547, 각 범주에서의 갯수: {'문화': 1065, '지역': 15380, '국제': 226, '정치': 2320, '경제': 9460, '사회': 7247, '스포츠': 3245, 'IT_과학': 604}
단어:  이날, 총 갯수: 17318, 각 범주에서의 갯수: {'문화':

In [49]:
find_common_words_and_remove(num_categories=7, com_counts=300)

상위 100개의 단어 확인
단어:   진행, 총 갯수: 31208, 각 범주에서의 갯수: {'문화': 2185, '지역': 15842, '국제': 82, '정치': 929, '경제': 6248, '사회': 4316, '스포츠': 549, 'IT_과학': 1057}
단어:   관계자, 총 갯수: 22464, 각 범주에서의 갯수: {'문화': 705, '지역': 11946, '국제': 98, '정치': 591, '경제': 4994, '사회': 3056, '스포츠': 303, 'IT_과학': 771}
단어:   대표, 총 갯수: 14928, 각 범주에서의 갯수: {'문화': 1040, '지역': 4527, '국제': 42, '정치': 3295, '경제': 3146, '사회': 1400, '스포츠': 929, 'IT_과학': 549}
단어:   시작, 총 갯수: 13749, 각 범주에서의 갯수: {'문화': 1225, '지역': 5546, '국제': 75, '정치': 714, '경제': 3275, '사회': 1933, '스포츠': 381, 'IT_과학': 600}
총 4개의 단어가 겹칩니다.


In [ ]:
from collections import Counter

def count_and_remove_low_occurrence_keywords(train_df, test_df, threshold=3):
    # '키워드' 열의 각 값을 쉼표로 분리하여 리스트로 변환
    train_df['키워드_리스트'] = train_df['키워드_통합'].apply(lambda x: x.split(','))
    test_df['키워드_리스트'] = test_df['키워드_통합'].apply(lambda x: x.split(','))

    # '분류_대분류' 별로 단어들을 추출하고 카운팅
    category_keywords = {category: [] for category in train_df["분류_대분류"].unique()}

    for category in category_keywords.keys():
        words = train_df.loc[train_df["분류_대분류"] == category, "키워드_리스트"].tolist()
        category_keywords[category] = [word.strip() for sublist in words for word in sublist if word.strip()]

    # 각 '분류_대분류' 별로 단어들을 카운팅하고 단어가 threshold 이하로 존재하는 경우를 찾기
    low_occurrence_words = set()
    for category, words in category_keywords.items():
        word_counts = Counter(words)
        low_occurrence_words.update({word for word, count in word_counts.items() if count <= threshold})

    # 단어가 threshold 이하로 존재하는 경우를 '키워드' 열에서 제거
    def remove_low_occurrence_words(keywords):
        return ', '.join([word.strip() for word in keywords.split(',') if word.strip() and word.strip() not in low_occurrence_words])

    train_df['키워드_통합'] = train_df['키워드_통합'].apply(remove_low_occurrence_words)

    # test_df에서도 동일한 단어를 제거
    def remove_low_occurrence_words_from_test(keywords):
        return ', '.join([word.strip() for word in keywords.split(',') if word.strip() and word.strip() not in low_occurrence_words])

    test_df['키워드_통합'] = test_df['키워드_통합'].apply(remove_low_occurrence_words_from_test)

    # 제거 후 각 '분류_대분류' 별로 하위 10개의 단어와 그 갯수를 출력
    for category in category_keywords.keys():
        words = train_df.loc[train_df["분류_대분류"] == category, "키워드_통합"].apply(lambda x: x.split(',')).tolist()
        words = [word.strip() for sublist in words for word in sublist if word.strip()]
        word_counts = Counter(words)
        bottom_keywords = word_counts.most_common()[:-11:-1]
        print(f"분류_대분류: {category}")
        for word, count in bottom_keywords:
            print(f"  단어: {word}, 갯수: {count}")
        print()

In [ ]:
# 함수 실행
count_and_remove_low_occurrence_keywords(train_df, test_df, threshold=2)

In [12]:
# # '키워드' 열의 모든 단어들을 리스트로 모으기
# all_keywords = []
# train_df['키워드'].apply(lambda x: all_keywords.extend(x.split(',')))

# # 모든 단어들의 빈도 계산
# keyword_counts = pd.Series(all_keywords).value_counts()

# # 갯수가 1개 이하인 키워드들 필터링
# low_frequency_keywords = keyword_counts[keyword_counts <= 1].index.tolist()
# print("\n갯수가 1개 이하인 키워드들:")
# low_frequency_keywords

In [13]:
# # '증오'가 정확히 일치하는 행 필터링
# filtered_df = train_df[train_df['키워드'].str.contains(r'\b증오\b', na=False)]

# # 결과 출력
# filtered_df.iloc[3]['키워드']

'이재명, 큰절, 눈물, 상처, 이재명, 더불어민주당, 대선, 후보, 연일, 반성, 사죄, 자신, 약점, 도덕성, 민주당, 사과, 중도층, 지지, 대선, 최대, 분기점, 연휴, 후보, 상태, 지지율, 정체, 타개, 위기감, 작용, 후보, 경기, 성남, 상대원, 시장, 자신, 유년기, 시절, 언급, 눈물, 자리, 이낙연, 민주당, 대표, 동행, 후보, 초등학교, 산꼭대기, 어머니, 화장실, 출근, 공장, 행복, 눈시울, 후보, 아버지, 시장, 청소, 노동자, 어머니, 시장, 공중, 화장실, 이용자, 소변, 대변, 설명, 화장실, 아들, 어머니, 거짓말, 판검사, 실력, 변호사, 노력, 자리, 상처, 토로, 후보, 형수, 욕설, 가족, 논란, 고개, 민주당, 반성, 강조, 후보, 경기, 용인시, 경기도, 공약, 윤호중, 원내, 대표, 민주당, 의원, 예정, 큰절, 후보, 사죄, 큰절, 후보, 민주당, 개혁, 진보, 세력, 핵심, 가치, 공정, 측면, 부족, 국민들, 남불, 질책, 생각, 관계자, 민주당, 선대위, 선대위, 현상, 지지율, 정체, 위기의식, 후보, 차원, 설명, 후보, 국민, 주장, 세대포위론, 적극, 반박, 세대포용, 세대포위론, 전폭적, 지지, 부모, 세대, 지지, 전략, 후보, 경기, 유세, 국민, 겨냥, 염장, 이익, 타인, 고통, 강요, 분열, 증오, 이용, 포위, 세대포위론, 비판, 박재현'

In [14]:
# # '키워드' 열의 모든 단어들을 리스트로 모으기
# all_keywords = []
# train_df['키워드'].apply(lambda x: all_keywords.extend(x.split(',')))

# # 모든 단어들의 빈도 계산
# keyword_counts = pd.Series(all_keywords).value_counts()

# # 갯수가 2개 이하인 키워드들 필터링
# low_frequency_keywords = keyword_counts[keyword_counts == 2].index.tolist()
# print("\n갯수가 2개인 키워드들:")
# low_frequency_keywords

In [ ]:
# import pandas as pd

# # '키워드' 열의 모든 단어들을 리스트로 모으기
# all_keywords = []
# train_df['키워드'].apply(lambda x: all_keywords.extend(x.split(',')))

# # 모든 단어들의 빈도 계산
# keyword_counts = pd.Series(all_keywords).value_counts()

# # 갯수가 1개 이하인 키워드들 필터링
# low_frequency_keywords = keyword_counts[keyword_counts <= 1].index.tolist()
# print("\n갯수가 1개 이하인 키워드들:")
# print(low_frequency_keywords)

# # 갯수가 1개 이하인 키워드들을 제거하는 함수
# def remove_low_frequency_keywords(keywords):
#     valid_keywords = [word.strip() for word in keywords.split(',') if word.strip() not in low_frequency_keywords]
#     return ', '.join(valid_keywords)

# # 원본 데이터에서 해당 키워드들을 제거
# train_df['키워드'] = train_df['키워드'].apply(remove_low_frequency_keywords)
# test_df['키워드'] = test_df['키워드'].apply(remove_low_frequency_keywords)

# # 결과 출력
# print("\n수정된 데이터프레임:")
# print(train_df)

In [ ]:
# import pandas as pd

# # '키워드' 열의 모든 단어들을 리스트로 모으기
# all_keywords = []
# train_df['키워드'].apply(lambda x: all_keywords.extend(x.split(',')))

# # 모든 단어들의 빈도 계산
# keyword_counts = pd.Series(all_keywords).value_counts()

# # 갯수가 2개인 키워드들 필터링
# low_frequency_keywords = keyword_counts[keyword_counts == 2].index.tolist()
# print("\n갯수가 2개인 키워드들:")
# print(low_frequency_keywords)

# # 갯수가 2개인 키워드들을 제거하는 함수
# def remove_low_frequency_keywords(keywords):
#     valid_keywords = [word.strip() for word in keywords.split(',') if word.strip() not in low_frequency_keywords]
#     return ', '.join(valid_keywords)

# # 원본 데이터에서 해당 키워드들을 제거
# train_df['키워드'] = train_df['키워드'].apply(remove_low_frequency_keywords)
# test_df['키워드'] = test_df['키워드'].apply(remove_low_frequency_keywords)

# # 결과 출력
# print("\n수정된 데이터프레임:")
# print(train_df)